In [2]:
# GEE Analysis with FastAPI Integration and MapStore Catalog
# This notebook demonstrates:
# 1. GEE Authentication using GCP service account
# 2. Creating cloudless Sentinel-2 imagery composite
# 3. Pushing results to FastAPI service
# 4. Making layers available for MapStore catalog

import ee
import json
import os
import sys
import requests
from datetime import datetime
import folium

# Add gee_lib to path
sys.path.insert(0, '/usr/src/app/gee_lib')

print("✓ Imports loaded successfully")

✓ Imports loaded successfully


In [3]:
# Step 1: Initialize Google Earth Engine with Service Account
def initialize_gee():
    """Initialize GEE using service account credentials"""
    try:
        # Load credentials from user_id.json
        credentials_path = '/usr/src/app/user_id.json'
        
        with open(credentials_path, 'r') as f:
            credentials_data = json.load(f)
        
        service_account = credentials_data['client_email']
        
        # Initialize Earth Engine
        credentials = ee.ServiceAccountCredentials(service_account, credentials_path)
        ee.Initialize(credentials)
        
        print(f"✓ GEE Initialized successfully")
        print(f"  Service Account: {service_account}")
        print(f"  Project ID: {credentials_data['project_id']}")
        
        return True
    except Exception as e:
        print(f"✗ Error initializing GEE: {e}")
        return False

# Initialize GEE
initialize_gee()


✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan


True

In [4]:
# Step 2: Define Area of Interest (AOI)
# Example: A region in Indonesia (Kalimantan)
aoi_coords = [
    [109.5, -1.5],
    [110.5, -1.5],
    [110.5, -0.5],
    [109.5, -0.5],
    [109.5, -1.5]
]

aoi = ee.Geometry.Polygon(aoi_coords)
aoi_center = aoi.centroid().coordinates().getInfo()

print("✓ AOI defined")
print(f"  Center coordinates: {aoi_center}")
print(f"  Area: {aoi.area().divide(1e6).getInfo():.2f} km²")


✓ AOI defined
  Center coordinates: [109.99999999999977, -1.000012676912049]
  Area: 12362.61 km²


In [5]:
# Step 3: Create Cloudless Sentinel-2 Composite using gee_lib
from osi.image_collection.main import ImageCollection

# Configuration
config = {
    'IsThermal': False,
}

date_start_end = ['2023-01-01', '2023-12-31']
cloud_cover_threshold = 20  # Maximum cloud cover percentage

print("Creating cloudless Sentinel-2 composite...")
print(f"  Date range: {date_start_end[0]} to {date_start_end[1]}")
print(f"  Cloud cover threshold: {cloud_cover_threshold}%")

# Create ImageCollection instance
img_collection = ImageCollection(
    I_satellite='Sentinel',
    region='asia',
    AOI=aoi,
    date_start_end=date_start_end,
    cloud_cover_threshold=cloud_cover_threshold,
    config=config
)

# Get cloud-masked image collection
sentinel_collection = img_collection.image_collection_mask()

# Create median composite (cloudless)
sentinel_composite = img_collection.image_mosaick()

print("✓ Sentinel-2 cloudless composite created")
print(f"  Number of images used: {sentinel_collection.size().getInfo()}")


Creating cloudless Sentinel-2 composite...
  Date range: 2023-01-01 to 2023-12-31
  Cloud cover threshold: 20%
selecting Sentinel images
selecting Sentinel images
✓ Sentinel-2 cloudless composite created
  Number of images used: 25


In [6]:
# Step 4: Generate Analysis Products
# Create True Color, False Color, and NDVI visualizations

# True Color RGB (Natural Color)
true_color = sentinel_composite.select(['red', 'green', 'blue'])

# False Color (NIR, Red, Green) - highlights vegetation
false_color = sentinel_composite.select(['nir', 'red', 'green'])

# Calculate NDVI
ndvi = sentinel_composite.normalizedDifference(['nir', 'red']).rename('NDVI')

# Calculate EVI (Enhanced Vegetation Index)
evi = sentinel_composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': sentinel_composite.select('nir'),
        'RED': sentinel_composite.select('red'),
        'BLUE': sentinel_composite.select('blue')
    }
).rename('EVI')

# Calculate NDWI (Normalized Difference Water Index)
ndwi = sentinel_composite.normalizedDifference(['green', 'nir']).rename('NDWI')

print("✓ Analysis products generated:")
print("  - True Color RGB")
print("  - False Color Composite")
print("  - NDVI (Normalized Difference Vegetation Index)")
print("  - EVI (Enhanced Vegetation Index)")
print("  - NDWI (Normalized Difference Water Index)")


✓ Analysis products generated:
  - True Color RGB
  - False Color Composite
  - NDVI (Normalized Difference Vegetation Index)
  - EVI (Enhanced Vegetation Index)
  - NDWI (Normalized Difference Water Index)


In [7]:
# Step 5: Create GEE Map IDs for Tile Serving
# These Map IDs can be used to serve tiles via URLs

# Visualization parameters
vis_params = {
    'true_color': {
        'bands': ['red', 'green', 'blue'],
        'min': 0,
        'max': 0.3,
        'gamma': 1.4
    },
    'false_color': {
        'bands': ['nir', 'red', 'green'],
        'min': 0,
        'max': 0.5,
        'gamma': 1.4
    },
    'ndvi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['red', 'yellow', 'green', 'darkgreen']
    },
    'evi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['brown', 'yellow', 'lightgreen', 'darkgreen']
    },
    'ndwi': {
        'min': -0.3,
        'max': 0.3,
        'palette': ['white', 'lightblue', 'blue', 'darkblue']
    }
}

# Generate Map IDs
print("Generating GEE Map IDs...")

map_ids = {}
map_ids['true_color'] = true_color.getMapId(vis_params['true_color'])
map_ids['false_color'] = false_color.getMapId(vis_params['false_color'])
map_ids['ndvi'] = ndvi.getMapId(vis_params['ndvi'])
map_ids['evi'] = evi.getMapId(vis_params['evi'])
map_ids['ndwi'] = ndwi.getMapId(vis_params['ndwi'])

print("✓ Map IDs generated for all layers")

# Display tile URLs
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")


Generating GEE Map IDs...
✓ Map IDs generated for all layers

TRUE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/d0a671ae85a1b78c9e9c444dd5f2022e-3dc09c9d6b61a9b48241242d54a8e3b7/tiles/{z}/{x}/{y}

FALSE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/f605a414e23d07e0dd1d973ef3908bc6-35588a5f7d1c176daab379c5424826e7/tiles/{z}/{x}/{y}

NDVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/d8d417a6cdbe875968eead89959d1199-811ce37bda82a6be5fa21fce1edc9358/tiles/{z}/{x}/{y}

EVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/745202459b174d7fa6fe999a32752a7a-701efeb7934684e31deda1fe295cf6c1/tiles/{z}/{x}/{y}

NDWI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/e72d0a0c4821e48eef04bc16539eb613-9b4a3cce6c5c78bcee89e84b5cf89598/tiles/{z}/{x}/{y}


In [8]:
# Step 6: Visualize on Interactive Map using Folium
# Create an interactive map centered on AOI

# Create base map
center_lat, center_lon = aoi_center[1], aoi_center[0]
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add GEE tile layers
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    folium.TileLayer(
        tiles=tile_url,
        attr='Google Earth Engine',
        name=layer_name.replace('_', ' ').title(),
        overlay=True,
        control=True
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

print("✓ Interactive map created with all layers")
print(f"  Map centered at: ({center_lat:.4f}, {center_lon:.4f})")

# Display the map
m


✓ Interactive map created with all layers
  Map centered at: (-1.0000, 110.0000)


In [9]:
# Step 7: Prepare Data for FastAPI Service
# Create a payload with layer information and metadata

project_id = f"sentinel_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Prepare layer data for FastAPI
layers_data = {
    'project_id': project_id,
    'project_name': 'Sentinel-2 Cloudless Composite Analysis',
    'aoi': {
        'type': 'Polygon',
        'coordinates': [aoi_coords],
        'center': aoi_center
    },
    'date_range': {
        'start': date_start_end[0],
        'end': date_start_end[1]
    },
    'analysis_params': {
        'satellite': 'Sentinel-2',
        'cloud_cover_threshold': cloud_cover_threshold,
        'image_count': sentinel_collection.size().getInfo()
    },
    'layers': {}
}

# Add layer information
for layer_name, map_id_dict in map_ids.items():
    layers_data['layers'][layer_name] = {
        'name': layer_name.replace('_', ' ').title(),
        'description': f'{layer_name.upper()} visualization from Sentinel-2',
        'tile_url': map_id_dict['tile_fetcher'].url_format,
        'map_id': map_id_dict['mapid'],
        'token': map_id_dict['token'],
        'vis_params': vis_params[layer_name]
    }

print("✓ Data prepared for FastAPI")
print(f"  Project ID: {project_id}")
print(f"  Total layers: {len(layers_data['layers'])}")


✓ Data prepared for FastAPI
  Project ID: sentinel_analysis_20251021_145859
  Total layers: 5


In [10]:
# Step 8: Push Results to FastAPI Service
# Send the analysis results to FastAPI so they can be served to MapStore

# FastAPI service URL (within Docker network)
fastapi_url = "http://fastapi:8000"

def push_to_fastapi(data, endpoint="/layers/register"):
    """Push GEE analysis results to FastAPI service"""
    try:
        # Send POST request to FastAPI
        url = f"{fastapi_url}{endpoint}"
        print(f"Pushing to: {url}")
        
        response = requests.post(
            url,
            json=data,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✓ Successfully pushed to FastAPI")
            print(f"  Status: {result.get('status')}")
            print(f"  Project ID: {result.get('project_id')}")
            print(f"  Layers Count: {result.get('layers_count')}")
            print(f"  Message: {result.get('message')}")
            return result
        else:
            print(f"✗ Error: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"  Detail: {error_detail.get('detail', response.text)}")
            except:
                print(f"  Response: {response.text}")
            return None
    except requests.exceptions.ConnectionError as e:
        print(f"✗ Connection Error: Cannot reach FastAPI service")
        print(f"  Make sure FastAPI container is running")
        print(f"  Check: docker ps | grep fastapi")
        return None
    except Exception as e:
        print(f"✗ Error pushing to FastAPI: {e}")
        import traceback
        print(f"  Traceback: {traceback.format_exc()}")
        return None

# Push data to FastAPI (using the new register endpoint)
print("Registering layers with FastAPI...")
result = push_to_fastapi(layers_data)

if result:
    print("\n📡 Data is now available in FastAPI service")
    print(f"   Access layers: {fastapi_url}/layers/{project_id}")
    print(f"   FastAPI Docs: {fastapi_url}/docs")
else:
    print("\n⚠️  Failed to push to FastAPI")
    print("   Layers are still available via direct GEE tile URLs")
    print("   You can still add them to MapStore manually")


Registering layers with FastAPI...
Pushing to: http://fastapi:8000/layers/register
✓ Successfully pushed to FastAPI
  Status: success
  Project ID: sentinel_analysis_20251021_145859
  Layers Count: 5
  Message: Layers registered successfully

📡 Data is now available in FastAPI service
   Access layers: http://fastapi:8000/layers/sentinel_analysis_20251021_145859
   FastAPI Docs: http://fastapi:8000/docs


In [11]:
# Step 9: Generate MapStore Catalog Configuration
# Create configuration that MapStore can use to add these layers to its catalog

mapstore_config = {
    'catalog': {
        'services': {
            'GEE_Sentinel_Analysis': {
                'type': 'tile',
                'title': f'Sentinel-2 Analysis - {project_id}',
                'description': 'Cloudless Sentinel-2 composite and derived indices',
                'url': fastapi_url,
                'layers': []
            }
        }
    },
    'layers': []
}

# Add each layer to MapStore configuration
for layer_name, layer_info in layers_data['layers'].items():
    # Format layer for MapStore
    mapstore_layer = {
        'type': 'tile',
        'name': f'{project_id}_{layer_name}',
        'title': layer_info['name'],
        'description': layer_info['description'],
        'url': layer_info['tile_url'],
        'visibility': False,  # Start hidden, user can toggle
        'opacity': 1.0,
        'format': 'image/png',
        'transparent': True,
        'tileSize': 256,
        'metadata': {
            'analysis_date': datetime.now().isoformat(),
            'source': 'Google Earth Engine',
            'satellite': 'Sentinel-2',
            'date_range': f"{date_start_end[0]} to {date_start_end[1]}"
        }
    }
    
    mapstore_config['layers'].append(mapstore_layer)
    mapstore_config['catalog']['services']['GEE_Sentinel_Analysis']['layers'].append(layer_name)

print("✓ MapStore catalog configuration generated")
print(f"  Service name: GEE_Sentinel_Analysis")
print(f"  Total layers for catalog: {len(mapstore_config['layers'])}")

# Display the configuration
print("\n📋 MapStore Configuration:")
print(json.dumps(mapstore_config, indent=2))


✓ MapStore catalog configuration generated
  Service name: GEE_Sentinel_Analysis
  Total layers for catalog: 5

📋 MapStore Configuration:
{
  "catalog": {
    "services": {
      "GEE_Sentinel_Analysis": {
        "type": "tile",
        "title": "Sentinel-2 Analysis - sentinel_analysis_20251021_145859",
        "description": "Cloudless Sentinel-2 composite and derived indices",
        "url": "http://fastapi:8000",
        "layers": [
          "true_color",
          "false_color",
          "ndvi",
          "evi",
          "ndwi"
        ]
      }
    }
  },
  "layers": [
    {
      "type": "tile",
      "name": "sentinel_analysis_20251021_145859_true_color",
      "title": "True Color",
      "description": "TRUE_COLOR visualization from Sentinel-2",
      "url": "https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/d0a671ae85a1b78c9e9c444dd5f2022e-3dc09c9d6b61a9b48241242d54a8e3b7/tiles/{z}/{x}/{y}",
      "visibility": false,
      "opacity": 1.0,
      "form

In [12]:
# Step 10: Test Direct Access to Layers
# Verify that the layers are accessible through FastAPI

def test_layer_access(project_id, layer_name):
    """Test if layer is accessible through FastAPI"""
    try:
        # Get layer information
        response = requests.get(
            f"{fastapi_url}/layers/{project_id}",
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✓ Layer accessible: {layer_name}")
            print(f"  Available layers: {list(result.get('layers', {}).keys())}")
            return True
        else:
            print(f"✗ Layer not accessible: {response.status_code}")
            return False
    except Exception as e:
        print(f"✗ Error testing access: {e}")
        return False

# Test access
print("Testing layer access through FastAPI...")
test_layer_access(project_id, 'ndvi')


Testing layer access through FastAPI...
✓ Layer accessible: ndvi
  Available layers: ['true_color', 'false_color', 'ndvi', 'evi', 'ndwi']


True

localhost:8888

🔧 Creating WMS endpoints for FastAPI service...
🔧 Creating WMS endpoints for FastAPI service...
✅ FastAPI WMS endpoints created successfully
  File: /usr/src/app/data/sentinel_analysis_20251021_145859_fastapi_wms_endpoints.py
\n📋 WMS Endpoints for FastAPI:
  GetCapabilities: http://localhost:8001/wms?service=WMS&version=1.3.0&request=GetCapabilities
  GetMap: http://localhost:8001/wms?service=WMS&version=1.3.0&request=GetMap&layers={layer_name}&bbox={bbox}&width=512&height=512&crs=EPSG:4326&format=image/png
\n🚀 How to Add to Your FastAPI Service:
  1. Copy the code from: /usr/src/app/data/sentinel_analysis_20251021_145859_fastapi_wms_endpoints.py
  2. Add it to your fastapi-gee-service/main.py file
  3. Restart your FastAPI service
  4. Test: http://localhost:8001/wms?service=WMS&version=1.3.0&request=GetCapabilities
\n💡 Benefits of Using FastAPI Directly:
  ✅ No separate service needed
  ✅ Uses existing authentication and middleware
  ✅ Integrated with your current architecture
  ✅ Si

In [14]:
# BONUS: Save configuration to file for later use
output_file = f'/usr/src/app/data/{project_id}_config.json'

try:
    # Ensure data directory exists
    os.makedirs('/usr/src/app/data', exist_ok=True)
    
    # Prepare complete configuration
    complete_config = {
        'project_info': layers_data,
        'mapstore_config': mapstore_config,
        'generated_at': datetime.now().isoformat()
    }
    
    # Save to file
    with open(output_file, 'w') as f:
        json.dump(complete_config, f, indent=2)
    
    print(f"✓ Configuration saved to: {output_file}")
    print(f"  File size: {os.path.getsize(output_file)} bytes")
    print(f"\n  You can use this file to:")
    print(f"  1. Import layers into MapStore")
    print(f"  2. Share analysis configuration with team")
    print(f"  3. Reproduce analysis with same parameters")
except Exception as e:
    print(f"✗ Error saving configuration: {e}")


✓ Configuration saved to: /usr/src/app/data/sentinel_analysis_20251020_174835_config.json
  File size: 8206 bytes

  You can use this file to:
  1. Import layers into MapStore
  2. Share analysis configuration with team
  3. Reproduce analysis with same parameters
